<a href="https://colab.research.google.com/github/MuayThaiLegz/PracticeCrazy/blob/main/OaklandPromiseDataTools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install sweetviz pygwalker

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO


import sweetviz as sv

def create_report(df):
  return sv.analyze(df)

from google.colab import data_table
data_table.enable_dataframe_formatter()

# California Department of Education (CDE) DataQuest:

For now I am storing the data into csv files but we could just as easy store the processed data into any database.

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

def download_and_process_data(url, output_file):
    try:
        session = requests.Session()
        session.max_redirects = 10  # Set a higher redirect limit or adjust as necessary
        print(f"Accessing {url}")
        response = session.get(url, allow_redirects=False)

        if response.status_code in [301, 302]:
            redirect_url = response.headers['Location']
            print(f"Redirected to: {redirect_url}")
            response = session.get(redirect_url)

        response.raise_for_status()  # Check for request errors
        print(f"Successfully accessed {url}")
    except requests.exceptions.TooManyRedirects as e:
        print(f"Too many redirects: {e}")
        return
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table")

    if not table:
        print(f"No table found on the page: {url}")
        return

    rows = table.find_all("tr")
    all_data = []

    for row in rows[1:]:
        columns = row.find_all("td")
        if len(columns) < 2:  # Ensure there are enough columns
            continue

        year = columns[0].text.strip()
        file_link = columns[1].find("a")

        if file_link is None:
            print(f"No link found in row: {row}")
            continue

        file_url = file_link["href"]

        if not file_url.startswith("http"):
            file_url = requests.compat.urljoin(url, file_url)

        print(f"Attempting to download: {file_url}")

        try:
            file_response = session.get(file_url)
            file_response.raise_for_status()  # Check for request errors
        except requests.exceptions.TooManyRedirects as e:
            print(f"Too many redirects for {file_url}: {e}")
            continue
        except requests.exceptions.RequestException as e:
            print(f"An error occurred while downloading {file_url}: {e}")
            continue

        file_content = file_response.content

        try:
            df = pd.read_csv(StringIO(file_content.decode('utf-8')), delimiter='\t')
        except UnicodeDecodeError:
            df = pd.read_csv(StringIO(file_content.decode('ISO-8859-1')), delimiter='\t')
        except pd.errors.ParserError as e:
            print(f"Failed to parse the file {file_url}: {e}")
            continue

        df['Year'] = year
        all_data.append(df)

    if all_data:
        merged_df = pd.concat(all_data, ignore_index=True)
        merged_df.to_csv(output_file, index=False)
        print(f"Data saved to {output_file}")
    else:
        print("No data was downloaded.")

datasets = [
    ("https://www.cde.ca.gov/ds/ad/filesgrads.asp", "graduates_by_race_gender_2011_2017.csv"),
    ("https://www.cde.ca.gov/ds/ad/filesgrad.asp", "graduates_by_ethnicity_2011_2017.csv"),
    ("https://www.cde.ca.gov/ds/ad/filesdropouts.asp", "dropouts_by_race_gender_2011_2017.csv"),
    ("https://www.cde.ca.gov/ds/ad/fileshse.asp", "homeless_student_enrollment_data.csv"),
    ("https://www.cde.ca.gov/ds/ad/filesenrcum.asp", "cumulative_enrollment_2016_2023.csv"),
    ("https://www.cde.ca.gov/ds/ad/fileshistenr8122.asp", "enrollment_byschool_historic.csv")
]

for url, output_file in datasets:
    download_and_process_data(url, output_file)


Accessing https://www.cde.ca.gov/ds/ad/filesgrads.asp
Successfully accessed https://www.cde.ca.gov/ds/ad/filesgrads.asp
Attempting to download: https://dq.cde.ca.gov/dataquest/dlfile/dlfile.aspx?cLevel=All&cYear=2016-17&cCat=Graduates&cPage=filesgrads
Attempting to download: https://dq.cde.ca.gov/dataquest/dlfile/dlfile.aspx?cLevel=All&cYear=2015-16&cCat=Graduates&cPage=filesgrads
Attempting to download: https://dq.cde.ca.gov/dataquest/dlfile/dlfile.aspx?cLevel=All&cYear=2014-15&cCat=Graduates&cPage=filesgrads
Attempting to download: https://dq.cde.ca.gov/dataquest/dlfile/dlfile.aspx?cLevel=All&cYear=2013-14&cCat=Graduates&cPage=filesgrads
Attempting to download: https://dq.cde.ca.gov/dataquest/dlfile/dlfile.aspx?cLevel=All&cYear=2012-13&cCat=Graduates&cPage=filesgrads
Attempting to download: https://dq.cde.ca.gov/dataquest/dlfile/dlfile.aspx?cLevel=All&cYear=2011-12&cCat=Graduates&cPage=filesgrads
Attempting to download: https://dq.cde.ca.gov/dataquest/dlfile/dlfile.aspx?cLevel=All&cY

In [ ]:


files = {
    "graduates_by_race_gender": "graduates_by_race_gender_2011_2017.csv",
    "graduates_by_ethnicity": "graduates_by_ethnicity_2011_2017.csv",
    "dropouts_by_race_gender": "dropouts_by_race_gender_2011_2017.csv",
    "homeless_student_enrollment_data": "homeless_student_enrollment_data.csv",
    "cumulative_enrollment_2016_2023": "cumulative_enrollment_2016_2023.csv",
    "enrollment_byschool_historic": "enrollment_byschool_historic.csv"
}

dataframes = {name: pd.read_csv(filepath) for name, filepath in files.items()}

graduates_by_race_gender_df = dataframes["graduates_by_race_gender"]
graduates_by_ethnicity_df = dataframes["graduates_by_ethnicity"]
dropouts_by_race_gender_df = dataframes["dropouts_by_race_gender"]
homeless_student_enrollment_df = dataframes["homeless_student_enrollment_data"]
cumulative_enrollment_df = dataframes["cumulative_enrollment_2016_2023"]
enrollment_byschool_historic_df = dataframes["enrollment_byschool_historic"]



In [ ]:

Alameda_cumulative_enrollment =  cumulative_enrollment_df[cumulative_enrollment_df['CountyName'] == 'Alameda']
homeless_student_Alameda = homeless_student_enrollment_df[homeless_student_enrollment_df['County Name'] == 'Alameda']

cols_to_convert = [
    'Temporarily Doubled Up', 'Temporary Shelters', 'Hotels/Motels',
    'Temporarily Unsheltered', 'Missing/Unknown', 'Temporarily Doubled Up (percent)',
    'Temporary Shelters (percent)', 'Hotels/Motels (percent)',
    'Temporarily Unsheltered (percent)', 'Missing/Unknown (percent)'
]

for col in cols_to_convert:
    homeless_student_enrollment_df[col] = pd.to_numeric(homeless_student_enrollment_df[col], errors='coerce')



In [ ]:
selectACADEMIC_YEAR = list(enrollment_byschool_historic_df['ACADEMIC_YEAR'].unique())[:8]
select_enrollment_history = enrollment_byschool_historic_df[enrollment_byschool_historic_df['ACADEMIC_YEAR'].isin(selectACADEMIC_YEAR)].drop(columns=['DISTRICT_NAME','SCHOOL_NAME','Year'])
ALAMEDA_select_enrollment_history = select_enrollment_history[select_enrollment_history['COUNTY'] == 'ALAMEDA']
ALAMEDA_select_enrollment_history['ADULT'].fillna(ALAMEDA_select_enrollment_history['ADULT'].mean(), inplace=True)

updated_columns = [
    'Academic_Year', 'CDS_Code', 'County_Name', 'District_Name', 'School_Name',
    'Enrollment_Type', 'Race_Ethnicity', 'Gender',
    'Grade_K', 'Grade_1', 'Grade_2', 'Grade_3', 'Grade_4',
    'Grade_5', 'Grade_6', 'Grade_7', 'Grade_8', 'Ungraded_Elementary',
    'Grade_9', 'Grade_10', 'Grade_11', 'Grade_12', 'Ungraded_Secondary',
    'Total_Enrollment', 'Adult_Enrollment'
]

ALAMEDA_select_enrollment_history.columns = updated_columns

In [ ]:
# Convert specific columns related to homeless student enrollment and housing situations to numeric types for accurate analysis, handling any conversion errors by coercing them into NaN.



# print("Post numeric convertion:")
# homeless_student_enrollment_df.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))


#Enrollment by School

In [ ]:
print("\nEnrollment by School:")

ALAMEDA_select_enrollment_history.head()

In [ ]:
ALAMEDA_select_enrollment_history_report = create_report(ALAMEDA_select_enrollment_history)
ALAMEDA_select_enrollment_history_report.show_notebook()

# Graduates by Race and Gender

In [ ]:
print("\nGraduates by Race and Gender:")
graduates_by_race_gender_df.head()

In [ ]:
# display(graduates_by_race_gender_df.info(),
#         graduates_by_race_gender_df.describe())

In [ ]:
graduates_by_race_gender_report = create_report(graduates_by_race_gender_df)
graduates_by_race_gender_report.show_notebook()

# Dropouts by Race & Gender

In [ ]:
print("\nDropouts by Race & Gender:")
dropouts_by_race_gender_df.head()

In [ ]:
# display(dropouts_by_race_gender_df.info(),
#         dropouts_by_race_gender_df.describe())

In [ ]:
dropouts_by_race_gender_report = create_report(dropouts_by_race_gender_df)
dropouts_by_race_gender_report.show_notebook()

# Graduates by Ethnicity


In [ ]:
print("\nGraduates by Ethnicity:")
graduates_by_ethnicity_df.head()

In [ ]:
# display(graduates_by_ethnicity_df.info(),
#       graduates_by_ethnicity_df.describe())

In [ ]:
graduates_by_ethnicity_report = create_report(graduates_by_ethnicity_df)
graduates_by_ethnicity_report.show_notebook()

# Homeless Student Enrollment


In [ ]:
print("\nHomeless Student Enrollment:")
homeless_student_enrollment_df.head()

In [ ]:
# display(homeless_student_enrollment_df.info(),
#       homeless_student_enrollment_df.describe())

In [ ]:
homeless_student_enrollment_report = create_report(homeless_student_Alameda)
homeless_student_enrollment_report.show_notebook()

# Data Visualization

In [ ]:
# you can visualize any of the dataframe created above !

# homeless_student_enrollment_df, graduates_by_ethnicity_df, dropouts_by_race_gender_df,ALAMEDA_select_enrollment_history, graduates_by_race_gender_df

# Or filter / query specific data to visualize

# AlamedaSD = graduates_by_ethnicity_df[graduates_by_ethnicity_df['COUNTY'] == 'Alameda']

In [ ]:
import pygwalker as pyg

pyg.walk(ALAMEDA_select_enrollment_history)